In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import mmdet3d
from mmdet3d.models import build_model
from mmdet3d.datasets import build_dataloader, build_dataset
from mmcv import Config, DictAction
from mmcv.runner import load_checkpoint
import torch
from mmcv.parallel import MMDataParallel

In [3]:
cfg_path = "/project_data/ramanan/shubham/results/training/train_fcos3d_80_nov2/fcos3d_r101_caffe_fpn_gn-head_dcn_2x8_1x_nus-mono3d.py"
cfg = Config.fromfile(cfg_path)


In [4]:
ckpt_path = "/project_data/ramanan/shubham/results/training/train_fcos3d_80_nov2/latest.pth"

In [5]:
cfg.data.test

{'type': 'NuScenesMonoDataset',
 'data_root': '/project_data/ramanan/shubham/nuscenes/',
 'ann_file': '/project_data/ramanan/shubham/nuscenes/nuscenes_infos_val_mono3d.coco.json',
 'img_prefix': '/project_data/ramanan/shubham/nuscenes/',
 'classes': ['car',
  'truck',
  'trailer',
  'bus',
  'construction_vehicle',
  'bicycle',
  'motorcycle',
  'pedestrian',
  'traffic_cone',
  'barrier'],
 'pipeline': [{'type': 'LoadImageFromFileMono3D'},
  {'type': 'MultiScaleFlipAug',
   'scale_factor': 1.0,
   'flip': False,
   'transforms': [{'type': 'RandomFlip3D'},
    {'type': 'Normalize',
     'mean': [103.53, 116.28, 123.675],
     'std': [1.0, 1.0, 1.0],
     'to_rgb': False},
    {'type': 'Pad', 'size_divisor': 32},
    {'type': 'DefaultFormatBundle3D',
     'class_names': ['car',
      'truck',
      'trailer',
      'bus',
      'construction_vehicle',
      'bicycle',
      'motorcycle',
      'pedestrian',
      'traffic_cone',
      'barrier'],
     'with_label': False},
    {'type': 

In [6]:
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

In [7]:
cfg.data.test.test_mode = True
samples_per_gpu = cfg.data.test.pop('samples_per_gpu', 1)
distributed = False

In [8]:
samples_per_gpu

1

In [9]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=distributed,
        shuffle=False)

loading annotations into memory...
Done (t=5.79s)
creating index...
index created!


In [10]:
model = build_model(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, ckpt_path, map_location='cpu')

/home/shubham2/miniconda3/lib/python3.8/site-packages/mmdet/models/detectors/single_stage.py:28: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/home/shubham2/miniconda3/lib/python3.8/site-packages/mmdet/models/backbones/resnet.py:400: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


Use load_from_local loader


2021-11-27 03:21:16,184 - root - INFO - ModulatedDeformConvPack backbone.layer3.0.conv2 is upgraded to version 2.
2021-11-27 03:21:16,190 - root - INFO - ModulatedDeformConvPack backbone.layer3.1.conv2 is upgraded to version 2.
2021-11-27 03:21:16,194 - root - INFO - ModulatedDeformConvPack backbone.layer3.2.conv2 is upgraded to version 2.
2021-11-27 03:21:16,199 - root - INFO - ModulatedDeformConvPack backbone.layer3.3.conv2 is upgraded to version 2.
2021-11-27 03:21:16,204 - root - INFO - ModulatedDeformConvPack backbone.layer3.4.conv2 is upgraded to version 2.
2021-11-27 03:21:16,208 - root - INFO - ModulatedDeformConvPack backbone.layer3.5.conv2 is upgraded to version 2.
2021-11-27 03:21:16,212 - root - INFO - ModulatedDeformConvPack backbone.layer3.6.conv2 is upgraded to version 2.
2021-11-27 03:21:16,217 - root - INFO - ModulatedDeformConvPack backbone.layer3.7.conv2 is upgraded to version 2.
2021-11-27 03:21:16,221 - root - INFO - ModulatedDeformConvPack backbone.layer3.8.conv2 

In [11]:
if 'CLASSES' in checkpoint.get('meta', {}):
    model.CLASSES = checkpoint['meta']['CLASSES']
if 'PALETTE' in checkpoint.get('meta', {}):
    model.PALETTE = checkpoint['meta']['PALETTE']

In [12]:
model = MMDataParallel(model, device_ids=[0])
model = model.eval()

In [17]:
len(data_loader)

36114

In [13]:
for i, data in enumerate(data_loader):
    dat = data
    print(i,data.keys())
    with torch.no_grad():
        result = model(return_loss=False, rescale=True, **data)
    assert(False)

0 dict_keys(['img_metas', 'img'])


AssertionError: 

In [16]:
result

[{'img_bbox': {'boxes_3d': CameraInstance3DBoxes(
       tensor([[-8.2142e-02,  2.9642e+00,  5.6249e+00,  ...,  1.6000e+00,
            -1.2987e-02,  8.1857e-03],
           [ 5.7016e-02,  3.0957e+00,  1.4288e+01,  ...,  1.6134e+00,
            -6.4106e-03,  8.5495e-02],
           [-2.3318e-02,  3.0270e+00,  5.4026e+00,  ...,  1.5558e+00,
             2.9350e-02,  4.9544e-02],
           ...,
           [ 7.6450e+00,  1.4521e+00,  1.7494e+01,  ...,  2.4419e+00,
             1.2316e-03, -3.1850e-02],
           [ 7.6916e+00,  1.4624e+00,  1.7609e+01,  ...,  2.4586e+00,
            -6.0604e-03, -2.2411e-02],
           [ 7.8340e+00,  1.4763e+00,  1.7901e+01,  ...,  2.5883e+00,
            -4.3785e-03, -6.3289e-03]])),
   'scores_3d': tensor([0.3315, 0.2041, 0.1847, 0.1196, 0.1122, 0.1045, 0.0880, 0.0822, 0.0794,
           0.0768, 0.0738, 0.0688, 0.0681, 0.0661, 0.0604, 0.0572, 0.0566, 0.0562,
           0.0562, 0.0542, 0.0514, 0.0506, 0.0502, 0.0723, 0.0663, 0.0577, 0.0527,
           